In [6]:
import pandas as pd
import facebook_scraper
from facebook_scraper import get_posts
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from JH_Conn import DBConnection
import json
import os

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import os
import wget
import time

In [13]:
cred = pd.read_json(r'cred.json',typ='series')

In [12]:
# disable the browser notifications, using chrome options.
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# speciy the path
driver = webdriver.Chrome('chromedriver.exe',options=chrome_options)

# open the webpage
driver.get('https://www.facebook.com/')

In [14]:
# set username

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys(cred['email'])
password.clear()
password.send_keys(cred['key'])

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [115]:
#driver.get("https://m.facebook.com/jagoanhosting/posts/10159177562948588")
driver.get("https://m.facebook.com/jagoanhosting/posts/10159182504648588")


# get Comment (first stage)

In [116]:
class CommentStageOne:
    
    def getHTMLResult():
        res = driver.find_element_by_class_name('_45kb.ufi').get_attribute('innerHTML')
        res_b4 = BeautifulSoup(res, "html.parser")
    
        return res_b4
    
    def getCommentList(res_b4):
        comment_list = res_b4.find_all("div",{"class":"_2a_i"})
        
        comment_author = [k.find_all("div",{"class":"_2b05"})[0].text for k in comment_list]
        comment_author_url = [k.find_all("div",{"class":"_2b05"})[0].find_all("a",href=True)[0].get("href") for k in comment_list]
        comment_author_url_clean = [k.replace('?fref=nf&rc=p&__tn__=R','').replace('/','') for k in comment_author_url ]
        comment_author_comment = [k.select("[data-sigil='comment-body']")[0].text for k in comment_list]
        comment_author_id = [k.select("[data-sigil='comment-body']")[0].get("data-commentid") for k in comment_list]
    
        df_comment = pd.DataFrame([comment_author_id,comment_author,comment_author_url_clean,comment_author_comment]).T
        df_comment.columns = ['commentid','name','url','comment']
        
        return df_comment
    
    def getUrlCommentInsideComment(res_b4):
        
        def get_url_comment_inside_comment(comment_list_ke):
            row = comment_list_ke.select("[data-sigil='replies-see-more']")[0]
            row_str_2 = str(row).split("data-ajaxify-href=")[1].split("data-sigil")[0].replace("amp;",'').replace('"','')
            row_str_3 = 'https://mobile.facebook.com'+str(row_str_2)
            
            return row_str_3
        
        comment_list = res_b4.find_all("div",{"class":"_2a_i"})
        url_comment_in_comment = [get_url_comment_inside_comment(comment_list[k]) for k in range(len(comment_list))]
        df_comment['url_comment_in_comment'] = url_comment_in_comment
        
        return df_comment

In [117]:
res_b4 = CommentStageOne.getHTMLResult()

In [118]:
df_comment = CommentStageOne.getCommentList(res_b4)
df_comment

,commentid,name,url,comment
0,10159182504648588_10159182511908588,Joko Maryadi,ario.artettajaya,Ku kira .com
1,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2)
2,10159182504648588_10159182527503588,Naufaldwinasmi,naufaldwinasmi.naufaldwinasmi.7,Dikira .com 25rb
3,10159182504648588_10159182533413588,Muhamad Nurul Huda,huda.cahpmt,Ku kira .com (3)
4,10159182504648588_10159182541043588,Jaka Prasetya,jaka.prasetya.123276,Kukukuku kikikirarara perpanjangaaaaaaan hosti...
5,10159182504648588_10159182548623588,Mochamad Rizal,mochamadrizalprihatna,Ku kira. com (4)
6,10159182504648588_10159182563898588,Syarif Hidayat,syarhida,ku kira .gov
7,10159182504648588_10159182576923588,Muhamad Ari,muhamad.ari.99,Id ?
8,10159182504648588_10159182608248588,Rezza Indra Rahayu,rezzaidr,Ku kira .com (5)
9,10159182504648588_10159182616113588,Nuansa Dewangga,nuansadewangga,Tahun ke 2 berapaan ya itu


In [159]:
df_comment_with_url = CommentStageOne.getUrlCommentInsideComment(res_b4)
df_comment_with_url.reset_index(drop=True,inplace=True)
df_comment_with_url

,commentid,name,url,comment,url_comment_in_comment
0,10159182504648588_10159182511908588,Joko Maryadi,ario.artettajaya,Ku kira .com,https://mobile.facebook.com/comment/replies/?c...
1,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...
2,10159182504648588_10159182527503588,Naufaldwinasmi,naufaldwinasmi.naufaldwinasmi.7,Dikira .com 25rb,https://mobile.facebook.com/comment/replies/?c...
3,10159182504648588_10159182533413588,Muhamad Nurul Huda,huda.cahpmt,Ku kira .com (3),https://mobile.facebook.com/comment/replies/?c...
4,10159182504648588_10159182541043588,Jaka Prasetya,jaka.prasetya.123276,Kukukuku kikikirarara perpanjangaaaaaaan hosti...,https://mobile.facebook.com/comment/replies/?c...
5,10159182504648588_10159182548623588,Mochamad Rizal,mochamadrizalprihatna,Ku kira. com (4),https://mobile.facebook.com/comment/replies/?c...
6,10159182504648588_10159182563898588,Syarif Hidayat,syarhida,ku kira .gov,https://mobile.facebook.com/comment/replies/?c...
7,10159182504648588_10159182576923588,Muhamad Ari,muhamad.ari.99,Id ?,https://mobile.facebook.com/comment/replies/?c...
8,10159182504648588_10159182608248588,Rezza Indra Rahayu,rezzaidr,Ku kira .com (5),https://mobile.facebook.com/comment/replies/?c...
9,10159182504648588_10159182616113588,Nuansa Dewangga,nuansadewangga,Tahun ke 2 berapaan ya itu,https://mobile.facebook.com/comment/replies/?c...


# get Comment inside Comment

In [120]:
class CommentInsideComment: 
    
    def getHTMLResult():
        res2 = driver.find_element_by_class_name('_55x2._3u3w').get_attribute('innerHTML')
        res_b4_2 = BeautifulSoup(res2, "html.parser")
        
        return res_b4_2
    
    def getComment_author(res_b4_2):
        soup = res_b4_2.find_all("div",{"class":"_2b04"})
        author = [k.find("a",href=True).text for k in soup]
        
        return author
    
    def getComment_text(res_b4_2):
        soup = res_b4_2.select("[data-sigil='comment-body']")
        comment_text = [k.text for k in soup]
        
        return comment_text
        
    def getComment_id(res_b4_2):
        soup = res_b4_2.select("[data-sigil='comment-body']")
        comment_id = [k.get("data-commentid") for k in soup]
        
        return comment_id

In [121]:
ls_comment_inside_comment = []
for k in list(df_comment_with_url['url_comment_in_comment']):
    driver.get(k)
    res_b4_2 = CommentInsideComment.getHTMLResult()
    comment_author = CommentInsideComment.getComment_author(res_b4_2)
    comment_text = CommentInsideComment.getComment_text(res_b4_2)
    comment_id = CommentInsideComment.getComment_id(res_b4_2)
    comment_url = [k]*len(comment_id)
    
    hasil = [comment_id,comment_author,comment_text,comment_url]
    ls_comment_inside_comment.append(hasil)

In [181]:
df_comment_inside_comment = pd.concat([pd.DataFrame(k).T for k in ls_comment_inside_comment])
df_comment_inside_comment.columns = ['subcommentid','author','text','url_comment']

In [182]:
df_comment_inside_comment.reset_index(drop=True,inplace=True)

In [183]:
def extractCommentid(row):
    
    return row.split("ctoken=")[1].split("&count=")[0]

In [184]:
df_comment_inside_comment['commentid'] = df_comment_inside_comment['url_comment'].apply(extractCommentid)

In [199]:
df_comment_inside_comment[df_comment_inside_comment.commentid=='10159182504648588_10159182513808588']

,subcommentid,author,text,url_comment,commentid
1,10159182504648588_10159182514198588,Iman Anfrust,🎃,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588
2,10159182504648588_10159182530788588,PembuatJagoan Hosting Indonesia,Wafa Andharu Mimin ga bilang .com (2),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588
3,10159182504648588_10159185798903588,ARofiqi Maulana,Jagoan Hosting Indonesia .com nya dong min (3456),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588
4,10159182504648588_10159188762798588,PembuatJagoan Hosting Indonesia,ARofiqi Maulana uda sering nih Flash Sale .com...,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588


In [200]:
df_comment_with_url[df_comment_with_url.commentid=='10159182504648588_10159182513808588']

,commentid,name,url,comment,url_comment_in_comment
1,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...


In [192]:
dfMerge = pd.merge(df_comment_inside_comment,df_comment_with_url,
                   left_on='commentid',right_on='commentid',how='left')

In [201]:
df_comment_with_url.to_excel('df_comment_with_url.xlsx')

In [202]:
df_comment_inside_comment.to_excel('df_comment_inside_comment.xlsx')

In [198]:
dfMerge

,subcommentid,author,text,url_comment,commentid,name,url,comment,url_comment_in_comment
0,10159182504648588_10159182530328588,PembuatJagoan Hosting Indonesia,Joko Maryadi Mimin ga bilang .com loh,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182511908588,Joko Maryadi,ario.artettajaya,Ku kira .com,https://mobile.facebook.com/comment/replies/?c...
1,10159182504648588_10159182514198588,Iman Anfrust,🎃,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...
2,10159182504648588_10159182530788588,PembuatJagoan Hosting Indonesia,Wafa Andharu Mimin ga bilang .com (2),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...
3,10159182504648588_10159185798903588,ARofiqi Maulana,Jagoan Hosting Indonesia .com nya dong min (3456),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...
4,10159182504648588_10159188762798588,PembuatJagoan Hosting Indonesia,ARofiqi Maulana uda sering nih Flash Sale .com...,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...
5,10159182504648588_10159182711613588,PembuatJagoan Hosting Indonesia,Naufaldwinasmi 🙃 kenapa ga mau yang itu sih?,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182527503588,Naufaldwinasmi,naufaldwinasmi.naufaldwinasmi.7,Dikira .com 25rb,https://mobile.facebook.com/comment/replies/?c...
6,10159182504648588_10159182792373588,Naufaldwinasmi,Karena simple + enak diliat orang lain min 😂,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182527503588,Naufaldwinasmi,naufaldwinasmi.naufaldwinasmi.7,Dikira .com 25rb,https://mobile.facebook.com/comment/replies/?c...
7,10159182504648588_10159188763818588,PembuatJagoan Hosting Indonesia,Muhamad Nurul Huda Np: Amigdala-Ku Kira Kau Ru...,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182533413588,Muhamad Nurul Huda,huda.cahpmt,Ku kira .com (3),https://mobile.facebook.com/comment/replies/?c...
8,10159182504648588_10159188764563588,PembuatJagoan Hosting Indonesia,"Jaka Prasetya weittsss, ada yang beda nih dari...",https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182541043588,Jaka Prasetya,jaka.prasetya.123276,Kukukuku kikikirarara perpanjangaaaaaaan hosti...,https://mobile.facebook.com/comment/replies/?c...
9,10159182504648588_10159188857778588,Jaka Prasetya,550 km boleh tuh,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182541043588,Jaka Prasetya,jaka.prasetya.123276,Kukukuku kikikirarara perpanjangaaaaaaan hosti...,https://mobile.facebook.com/comment/replies/?c...


In [164]:
dfMerge['url_comment'][0]

'https://mobile.facebook.com/comment/replies/?ctoken=10159182504648588_10159182511908588&count=1&curr&pc=1&isinline&initcomp&ft_ent_identifier=10159182504648588&eav=AfYXyH9tWRH-YRDJRFd3HEOm_9ZYb9so7ez7y9pUAtFTtj7WSY6S0RHK_xXz-Ny0kQA&av=100064557612924&gfid=AQCyJy6yEnxcfg5PQ1Y&__tn__=R '

In [165]:
dfMerge['url_comment_in_comment'][0]

'https://mobile.facebook.com/comment/replies/?ctoken=10159182504648588_10159182511908588&count=1&curr&pc=1&isinline&initcomp&ft_ent_identifier=10159182504648588&eav=AfYXyH9tWRH-YRDJRFd3HEOm_9ZYb9so7ez7y9pUAtFTtj7WSY6S0RHK_xXz-Ny0kQA&av=100064557612924&gfid=AQCyJy6yEnxcfg5PQ1Y&__tn__=R '

In [145]:
cc = df_comment_with_url['url_comment_in_comment'][1]
cc

'https://mobile.facebook.com/comment/replies/?ctoken=10159182504648588_10159182513808588&count=4&curr&pc=1&isinline&initcomp&ft_ent_identifier=10159182504648588&eav=AfZWb5Ahd3awd7CTRHs_c434Wx5sDu3vQYTz8vZyvW-1r63nTUUauc7_TCtKRQWpSIY&av=100064557612924&gfid=AQDVXIxw-rO8OeTN0WQ&__tn__=R '

In [154]:
df2 = df_comment_inside_comment[df_comment_inside_comment.url_comment==cc]
df2

,comment_id,author,text,url_comment
1,10159182504648588_10159182514198588,Iman Anfrust,🎃,https://mobile.facebook.com/comment/replies/?c...
2,10159182504648588_10159182530788588,PembuatJagoan Hosting Indonesia,Wafa Andharu Mimin ga bilang .com (2),https://mobile.facebook.com/comment/replies/?c...
3,10159182504648588_10159185798903588,ARofiqi Maulana,Jagoan Hosting Indonesia .com nya dong min (3456),https://mobile.facebook.com/comment/replies/?c...
4,10159182504648588_10159188762798588,PembuatJagoan Hosting Indonesia,ARofiqi Maulana uda sering nih Flash Sale .com...,https://mobile.facebook.com/comment/replies/?c...


In [153]:
df1 = df_comment_with_url[df_comment_with_url.url_comment_in_comment==cc]
df1

,commentid,name,url,comment,url_comment_in_comment
1,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...


In [158]:
df1['url_comment_in_comment'][1]==df2['url_comment'][1]

True

In [150]:
pd.merge(df_comment_with_url,df_comment_inside_comment,left_on='url_comment_in_comment',right_on='url_comment',how='inner')

,commentid,name,url,comment,url_comment_in_comment,comment_id,author,text,url_comment
0,10159182504648588_10159182511908588,Joko Maryadi,ario.artettajaya,Ku kira .com,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182530328588,PembuatJagoan Hosting Indonesia,Joko Maryadi Mimin ga bilang .com loh,https://mobile.facebook.com/comment/replies/?c...
1,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182514198588,Iman Anfrust,🎃,https://mobile.facebook.com/comment/replies/?c...
2,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182530788588,PembuatJagoan Hosting Indonesia,Wafa Andharu Mimin ga bilang .com (2),https://mobile.facebook.com/comment/replies/?c...
3,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159185798903588,ARofiqi Maulana,Jagoan Hosting Indonesia .com nya dong min (3456),https://mobile.facebook.com/comment/replies/?c...
4,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159188762798588,PembuatJagoan Hosting Indonesia,ARofiqi Maulana uda sering nih Flash Sale .com...,https://mobile.facebook.com/comment/replies/?c...
5,10159182504648588_10159182527503588,Naufaldwinasmi,naufaldwinasmi.naufaldwinasmi.7,Dikira .com 25rb,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182711613588,PembuatJagoan Hosting Indonesia,Naufaldwinasmi 🙃 kenapa ga mau yang itu sih?,https://mobile.facebook.com/comment/replies/?c...
6,10159182504648588_10159182527503588,Naufaldwinasmi,naufaldwinasmi.naufaldwinasmi.7,Dikira .com 25rb,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159182792373588,Naufaldwinasmi,Karena simple + enak diliat orang lain min 😂,https://mobile.facebook.com/comment/replies/?c...
7,10159182504648588_10159182533413588,Muhamad Nurul Huda,huda.cahpmt,Ku kira .com (3),https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159188763818588,PembuatJagoan Hosting Indonesia,Muhamad Nurul Huda Np: Amigdala-Ku Kira Kau Ru...,https://mobile.facebook.com/comment/replies/?c...
8,10159182504648588_10159182541043588,Jaka Prasetya,jaka.prasetya.123276,Kukukuku kikikirarara perpanjangaaaaaaan hosti...,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159188764563588,PembuatJagoan Hosting Indonesia,"Jaka Prasetya weittsss, ada yang beda nih dari...",https://mobile.facebook.com/comment/replies/?c...
9,10159182504648588_10159182541043588,Jaka Prasetya,jaka.prasetya.123276,Kukukuku kikikirarara perpanjangaaaaaaan hosti...,https://mobile.facebook.com/comment/replies/?c...,10159182504648588_10159188857778588,Jaka Prasetya,550 km boleh tuh,https://mobile.facebook.com/comment/replies/?c...


In [151]:
df_comment_with_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   commentid               14 non-null     object
 1   name                    14 non-null     object
 2   url                     14 non-null     object
 3   comment                 14 non-null     object
 4   url_comment_in_comment  14 non-null     object
dtypes: object(5)
memory usage: 688.0+ bytes


In [152]:
df_comment_inside_comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   comment_id   22 non-null     object
 1   author       22 non-null     object
 2   text         22 non-null     object
 3   url_comment  22 non-null     object
dtypes: object(4)
memory usage: 832.0+ bytes


# Store Data

In [203]:
df_comment_with_url.head()

,commentid,name,url,comment,url_comment_in_comment
0,10159182504648588_10159182511908588,Joko Maryadi,ario.artettajaya,Ku kira .com,https://mobile.facebook.com/comment/replies/?c...
1,10159182504648588_10159182513808588,Wafa Andharu,Wafa.Andharu21,Ku kira .com (2),https://mobile.facebook.com/comment/replies/?c...
2,10159182504648588_10159182527503588,Naufaldwinasmi,naufaldwinasmi.naufaldwinasmi.7,Dikira .com 25rb,https://mobile.facebook.com/comment/replies/?c...
3,10159182504648588_10159182533413588,Muhamad Nurul Huda,huda.cahpmt,Ku kira .com (3),https://mobile.facebook.com/comment/replies/?c...
4,10159182504648588_10159182541043588,Jaka Prasetya,jaka.prasetya.123276,Kukukuku kikikirarara perpanjangaaaaaaan hosti...,https://mobile.facebook.com/comment/replies/?c...


In [ ]:
def StoreDataTotalLikesFanspage(datenow,data):
    sql_delete1 = "DELETE FROM fbscraper_total_fanspages_likes WHERE DATE(lastupdate) = DATE('"+str(datenow)+"')"
    cursor1 = connWarehouseWhmcs.cursor()
    cursor1.execute(sql_delete1)
    connWarehouseWhmcs.commit()

    sql_insert = "INSERT INTO fbscraper_total_fanspages_likes (username, total_likes, lastupdate  ) VALUES (%s,%s,%s)"    
    mycursor = connWarehouseWhmcs.cursor()
    mycursor.executemany(sql_insert,data.values.tolist())
    connWarehouseWhmcs.commit()